# Common Imports

In [ ]:
import faiss
import pickle
import numpy as np
from typing import List, Dict, Any, Optional

from start_utils import db_session

from dtos.services.etl.profile import ProfileData
from dtos.services.search.hard_criteria import HardCriteria

# ETL Load

In [256]:
from services.etl.load import ETLLoadService

In [193]:
file_path = "data/linkedin_data_subset.json"

etl_load_service = ETLLoadService(
    file_path=file_path
)

In [194]:
dataset = etl_load_service.run()

2025-07-30 16:14:38.497 | INFO     | services.etl.load:run:22 - Loading data from data/linkedin_data_subset.json...
July-30-2025 | 16:14:38 | INFO | Loading data from data/linkedin_data_subset.json... | services.etl.load:run:22 | {}
2025-07-30 16:27:33.968 | INFO     | services.etl.load:run:25 - Loaded 193796 profiles
July-30-2025 | 16:27:33 | INFO | Loaded 193796 profiles | services.etl.load:run:25 | {}


# ETL Extraction

In [254]:
from services.etl.extract import ETLExtractionService
etl_extract_service = ETLExtractionService(session=db_session)

In [255]:
profiles = etl_extract_service.run(profiles=dataset)

2025-07-30 17:31:50.899 | INFO     | services.etl.extract:run:252 - Extracting profiles: 193796
July-30-2025 | 17:31:50 | INFO | Extracting profiles: 193796 | services.etl.extract:run:252 | {}
100%|██████████| 193796/193796 [07:18<00:00, 441.98it/s] 
2025-07-30 17:39:09.381 | INFO     | services.etl.extract:run:379 - Profiles created: 193796
July-30-2025 | 17:39:09 | INFO | Profiles created: 193796 | services.etl.extract:run:379 | {}


# ETL Transform

In [252]:
from services.etl.transform import ETLTransformService
etl_transform_service = ETLTransformService()

In [253]:
documents = etl_transform_service.run(profiles=profiles)

2025-07-30 17:29:59.281 | INFO     | services.etl.transform:run:25 - Transforming 193796 profiles...
July-30-2025 | 17:29:59 | INFO | Transforming 193796 profiles... | services.etl.transform:run:25 | {}
2025-07-30 17:31:42.730 | INFO     | services.etl.transform:run:27 - Transformed 193796 profiles...
July-30-2025 | 17:31:42 | INFO | Transformed 193796 profiles... | services.etl.transform:run:27 | {}


# Vector Embedding of profiles

In [249]:
from services.vector_store.embedding import VectorStoreEmbeddingService

vector_store_embedding_service = VectorStoreEmbeddingService(
    model_name="voyage-3",
    max_concurrent_requests=100,
    use_progress=True,
    save_embeddings=True,
    embeddings_dir="embeddings",
)

embeddings = await vector_store_embedding_service.run_async(documents)

2025-07-30 17:25:12.804 | INFO     | services.vector_store.embedding:__init__:51 - Initialized VectorStoreEmbeddingService with max 100 concurrent requests


July-30-2025 | 17:25:12 | INFO | Initialized VectorStoreEmbeddingService with max 100 concurrent requests | services.vector_store.embedding:__init__:51 | {}
Embedding texts:   0%|          | 0/711 [00:00<?, ?it/s]2025-07-30 17:25:22.625 | ERROR    | services.vector_store.embedding:save_embeddings_to_json:187 - Error saving embeddings to JSON: Object of type ndarray is not JSON serializable
July-30-2025 | 17:25:22 | ERROR | Error saving embeddings to JSON: Object of type ndarray is not JSON serializable | services.vector_store.embedding:save_embeddings_to_json:187 | {}
Embedding texts:  14%|█▍        | 100/711 [00:09<00:59, 10.26it/s]2025-07-30 17:25:36.619 | ERROR    | services.vector_store.embedding:save_embeddings_to_json:187 - Error saving embeddings to JSON: Object of type ndarray is not JSON serializable
July-30-2025 | 17:25:36 | ERROR | Error saving embeddings to JSON: Object of type ndarray is not JSON serializable | services.vector_store.embedding:save_embeddings_to_json:187 | 

CancelledError: 

# Load Embeddings

In [ ]:
import os
import json
files = os.listdir("embeddings")
existing_dataset = []
for file in files:
    with open(os.path.join("embeddings", file), "r") as f:
        d = json.load(f)
        temp = list(filter(lambda x: x is not None, d))
        existing_dataset.extend(temp)

len(existing_dataset)

embeddings = []
for data in existing_dataset:
    embeddings.append(np.array(data.get("embedding")))

# Kmean Clustering on embeddings

In [247]:
from services.clustering.kmean import KMeansClusteringService

In [ ]:
kmeans_clustering_service = KMeansClusteringService(
    n_clusters=14,
)

kmean_cluster_labels, cluster_centers = kmeans_clustering_service.run(np.array(embeddings))

2025-07-30 14:27:30.354 | INFO     | __main__:run:161 - Creating 14 clusters...
July-30-2025 | 14:27:30 | INFO | Creating 14 clusters... | __main__:run:161 | {}


# Write indices to vector store

In [243]:
from services.vector_store.write import VectorStoreWriteService

In [244]:
vector_store_write_service = VectorStoreWriteService()

faiss_indices = vector_store_write_service.run(
    embeddings=np.array(embeddings),
    cluster_labels=kmean_cluster_labels,
)

2025-07-30 17:17:41.875 | INFO     | services.vector_store.write:run:87 - Running vector store write service for 193796 clusters
July-30-2025 | 17:17:41 | INFO | Running vector store write service for 193796 clusters | services.vector_store.write:run:87 | {}
2025-07-30 17:17:41.889 | INFO     | services.vector_store.write:run:93 - Writing embeddings for cluster 0
July-30-2025 | 17:17:41 | INFO | Writing embeddings for cluster 0 | services.vector_store.write:run:93 | {}
2025-07-30 17:17:41.961 | INFO     | services.vector_store.write:write:41 - Cluster 0: 17728 profiles, 17728 profile indices
July-30-2025 | 17:17:41 | INFO | Cluster 0: 17728 profiles, 17728 profile indices | services.vector_store.write:write:41 | {}
2025-07-30 17:17:42.059 | INFO     | services.vector_store.write:write:65 - Saved cluster 0 with 17728 embeddings
July-30-2025 | 17:17:42 | INFO | Saved cluster 0 with 17728 embeddings | services.vector_store.write:write:65 | {}
2025-07-30 17:17:42.062 | INFO     | services.

# Save System

In [272]:
def save_system(
    profiles: List[ProfileData],
    clusters: np.ndarray,
    cluster_centers: np.ndarray,
    embeddings: np.ndarray,
    faiss_indices: Dict[int, Dict[str, Any]],
    cluster_labels: np.ndarray,
    filepath: str,
):
    """
    Save the clustering system to disk
    """
    system_data = {
        'profiles': profiles,
        'clusters': clusters,
        'cluster_centers': cluster_centers,
        'embeddings': embeddings,
        'faiss_indices': faiss_indices,
        'cluster_labels': cluster_labels,
    }
    
    with open(filepath, 'wb') as f:
        pickle.dump(system_data, f)
    
    # Save FAISS indices separately
    for cluster_id, cluster_info in faiss_indices.items():
        faiss.write_index(cluster_info['index'], f"{filepath}_cluster_{cluster_id}.faiss")
    
    print(f"System saved to {filepath}")

In [274]:
save_system(
    profiles=profiles,
    clusters=kmean_cluster_labels,
    cluster_centers=kmeans_clustering_service.cluster_centers,
    embeddings=np.array(embeddings),
    faiss_indices=faiss_indices,
    filepath="data/clustering_system_v2.pkl",
    cluster_labels=kmean_cluster_labels
)

System saved to data/clustering_system_v2.pkl
